In [ ]:
import pandas as pd
import time
import json
import random
import numpy as np
import pickle
import os
import sqlite3
import psutil
import shutil
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from datetime import datetime
from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

def get_random_float(start=0.1, end=1.0):
    return np.random.uniform(start, end)

# PROXY = '150.241.92.165:57780'

# Instagram account
# USERNAME = 'shoshxm73ana_reeh'
# PASSWORD = 'BBn@#4545454646745'

# Options for Google Chrome
options = webdriver.ChromeOptions()
# options.add_argument("--no-sandbox")
# options.add_argument("--headless")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--blink-settings=imagesEnabled=false")
# Saving sessions and cookies
# options.add_argument("--user-data-dir=inst_sel3")
# options.add_argument("user-data-dir=inst_sel3")
options.add_argument("--incognito")
# options.add_argument(f"--proxy-server={PROXY}")
options.set_capability(
                        "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
                    )

driver = webdriver.Chrome(
    options=options
)


driver.set_window_size(1900, 1200)
print('='*20, '   Google Chrome Launched   ', '='*20)

driver.get('https://api.ipify.org/')
time.sleep(2)
# Set URL and waiting 3 seconds for page to load
url = 'https://www.tiktok.com/'
driver.get(url)
time.sleep(10)

# Устанавливаем соединение с базой данных
connection = sqlite3.connect('../videos_database.db')
cursor = connection.cursor()

# button_all = driver.find_elements(By.XPATH, ".//*[@class='css-1hs87dt-ButtonCategoryItemContainer e13i6o247']")
# buttons_themes = driver.find_elements(By.XPATH, ".//*[@class='css-12qul9x-ButtonCategoryItemContainer e13i6o247']")

# buttons = button_all + buttons_themes

def get_themes_buttons(driver):
    # button_all = driver.find_elements(By.XPATH, ".//*[@class='css-1hs87dt-ButtonCategoryItemContainer e13i6o247']")
    # buttons_themes = driver.find_elements(By.XPATH, ".//*[@class='css-12qul9x-ButtonCategoryItemContainer e13i6o247']")
    buttons_themes = driver.find_elements(By.CLASS_NAME, "e13i6o248")
    buttons_themes = [x for x in buttons_themes if 'Все' not in x.text]
    return buttons_themes

def create_requests_session(driver):
    s = requests.Session()
    # Set correct user agent
    selenium_user_agent = driver.execute_script("return navigator.userAgent;")
    s.headers.update({"user-agent": selenium_user_agent})
    
    for cookie in driver.get_cookies():
        s.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
        
    # retry = Retry(connect=3, backoff_factor=0.5)
    # adapter = HTTPAdapter(max_retries=retry)
    # s.mount('http://', adapter)
    # s.mount('https://', adapter)
        
    return s

videos_metadata = []
parsing_time = pd.to_datetime(datetime.now())
current_datetime = datetime.now().strftime('%Y%m%d_%H%M')

buttons_themes = get_themes_buttons(driver)
print('Buttons Founded... Proceeding')
for button in buttons_themes[:]:
    videos_cnt_before = len(videos_metadata)
    button_time1 = time.perf_counter()
    driver.execute_script("window.scrollTo(0, 0)")  # Scroll to top
    driver.execute_script("arguments[0].scrollIntoView();", button)
    time.sleep(3)
    button_text = button.text
    print('Processing video theme: {}'.format(button_text))
    try:
        button.click()
    except:
        driver.execute_script("arguments[0].scrollIntoView();", button)
        time.sleep(5)
        try:
            button.click()
        except:
            driver.execute_script("arguments[0].scrollIntoView();", button)
            time.sleep(5)
            try:
                button.click()
            except:
                try:
                    # arrow = driver.find_element(By.XPATH, ".//*[@class='css-brfiqi-DivArrowContainer e13i6o249']")
                    arrow = driver.find_elements(By.CLASS_NAME, "e13i6o2410")
                    arrow[-1].click()
                    time.sleep(2)
                    try:
                        button.click()
                    except:
                        try:
                            arrow = driver.find_elements(By.CLASS_NAME, "e13i6o2410")
                            arrow[-1].click()
                            time.sleep(2)
                        except:
                            print("Can't find the button")
                            break
                except:
                    print("Can't click on arrow")
                    break
    click_timestamp = pd.to_datetime(datetime.now().timestamp(), unit='s') - pd.Timedelta(seconds=5)
    time.sleep(7)
    print('click_timestamp_step')
    
    for i in range(20):
        # tiktoks = driver.find_elements(By.XPATH, ".//*[@class='css-x6y88p-DivItemContainerV2 e19c29qe7']")
        tiktoks = driver.find_elements(By.CLASS_NAME, "e19c29qe7")
        while len(tiktoks) == 0:
            time.sleep(5)
            tiktoks = driver.find_elements(By.CLASS_NAME, "e19c29qe7")
            
        driver.execute_script("arguments[0].scrollIntoView();", tiktoks[-1])
        time.sleep(get_random_float(2, 4))
    print('scrolling_down_step')
    log_entries = driver.get_log("performance")
    item_list_logs = []
    for log in log_entries:
        if 'item_list' in log['message'] and 'responseReceived' in log['message'] and pd.to_datetime(log['timestamp'], unit='ms') >= click_timestamp:
            item_list_logs.append(log)
    print('logs_get_step')
    # cursor.execute('''
    # select distinct vm.video_id
    # from videos_metadata_usa vm
    # ''')
    # column_names = [description[0] for description in cursor.description]
    # already_downloaded = pd.DataFrame(cursor.fetchall(), columns=column_names)
    already_downloaded = [x.replace('.mp4', '') for x in os.listdir('downloaded_tiktoks/') if '.mp4' in x]
            
    for item_list in item_list_logs:
        print('item_list_getting')
        request_session = create_requests_session(driver)
        full_log = json.loads(item_list['message'])
        endpoint_url = full_log['message']['params']['response']['url']
        
        try:
            print('creating_session_step')
            records = request_session.get(endpoint_url)
        except:
            print("Can't get itemList beacuse of Connection Refused")
            time.sleep(5)
            continue
        try:
            tiktoks_list = records.json()['itemList']
            print('videos_parsing_step')
            indices_to_drop = []
            idx = 0
            # videos_metadata.extend(tiktoks_list)
            for video in tiktoks_list:
                t1 = time.perf_counter()
                video_id = video['id']
                # if video_id in already_downloaded['video_id'].tolist():
                #     indices_to_drop.append(idx)
                #     continue
                if 'imagePost' in video.keys():
                    indices_to_drop.append(idx)
                    # time.sleep(1)
                    continue
                if video['video']['duration'] > 250:
                    indices_to_drop.append(idx)
                    # time.sleep(1)
                    continue
                
                # downloading_url = video['video']['playAddr']
                # for bitrateInfo in video['video']['bitrateInfo']:
                #     if bitrateInfo['GearName'] == 'lowest_540_0':
                #         downloading_url = bitrateInfo['PlayAddr']['UrlList'][0]
                #     else:
                downloading_url = video['video']['playAddr']
                if video_id in already_downloaded:
                    pass
                else:
                    try:
                        max_time_between_chunks = 30
                        chunk_size = 16*1024
                        # video_response = request_session.get(downloading_url, stream=True, timeout=(10, 30))
                        with request_session.get(downloading_url, stream=True, timeout=(10, 30)) as r:
                            with open('downloaded_tiktoks/{}.mp4'.format(video_id), 'wb') as f:
                                last_chunk_time = time.time()
                                for chunk in r.iter_content(chunk_size):
                                    if chunk:
                                        f.write(chunk)
                                        last_chunk_time = time.time()
                                    else:
                                        
                                        if time.time() - last_chunk_time > max_time_between_chunks:
                                            print("Can't download video")
                                            import os
                                            if os.path.exists('downloaded_tiktoks/{}.mp4'.format(video_id)):
                                                os.remove('downloaded_tiktoks/{}.mp4'.format(video_id))
                                            break

                        time.sleep(1)
                    except:
                        print("Can't download video... error with GET request")
                t2 = time.perf_counter()
                print('Video Processed in: {:.2f} seconds'.format(t2-t1))
                print('-'*30)
                idx += 1
            tiktoks_list = [tiktoks_list[i] for i in range(len(tiktoks_list)) if i not in indices_to_drop]
            for item in tiktoks_list:
                item['video_theme_button'] = button_text
                item['parsingTime'] = current_datetime
            videos_metadata.extend(tiktoks_list)
            time.sleep(get_random_float(1, 2))
        except:
            print("Can't process item_list, proceeding to the next")
                
    print('data_saving_step')
    videos_cnt_after = len(videos_metadata)
    new_videos_cnt = videos_cnt_after - videos_cnt_before
    
    print('--- Saving Data to JSON ---')
    print('--- Number of uplodaed videos: {} ---'.format(new_videos_cnt))
    with open('videos_metadata_json/videos_metadata_{}.json'.format(current_datetime), 'w') as output_file:
        json.dump(videos_metadata, output_file)
    print('--- JSON Saved ---')
    
    # print(all_videos_data['video_theme'].unique())
    buttons_themes = get_themes_buttons(driver)
    button_time2 = time.perf_counter()
    print('Theme Worktime: {:.2f} seconds'.format(button_time2 - button_time1))
    print('='*30)